In [58]:
import pandas as pd
import polars as pl
from timeit import timeit
import os, sys
print(os.getcwd())
os.chdir('/gpfs/data/healthcare-allocate/CLIF-MIMIC')
print(os.getcwd())

# proj_root = "/gpfs/data/healthcare-allocate/CLIF-MIMIC"
# if proj_root not in sys.path:
#     sys.path.append(proj_root)

from code.custom_utils import load_gz_csv, lookup_item

/gpfs/data/healthcare-allocate/CLIF-MIMIC
/gpfs/data/healthcare-allocate/CLIF-MIMIC


In [50]:
print(os.getcwd())

with open("/gpfs/data/healthcare-allocate/CLIF-MIMIC/code/custom_utils.py") as f:
    print(f.read())

/gpfs/data/healthcare-allocate/CLIF-MIMIC/code
import pandas as pd
import gzip

def load_gz_csv(file_path):
    with gzip.open(file_path, 'rt', encoding = 'utf-8') as f:
        df = pd.read_csv(f)
    return df

def lookup_item(df, kw: str, col: str = "label", case: bool = False):
    '''
    col = {"label", "abbr"}
    '''
    if col == "abbr": col == "abbreviation"
    output = df[
        df[col].str.contains(kw, case = case, na = False)
    ]
    
    return output


## hosp

In [2]:
d_labitems = pd.read_csv("mimic-iv-2.2/hosp/d_labitems.csv.gz")

In [4]:
# d_labitems.head(10)
contains = d_labitems["label"].str.contains("temp", case=False, na=False)
d_labitems[contains]

,itemid,label,fluid,category
23,50825,Temperature,Blood,Blood Gas


In [19]:
d_labitems["fluid"].unique()
d_labitems["category"].unique()

array(['Blood Gas', 'Chemistry', 'Hematology'], dtype=object)

In [12]:
omr = pd.read_csv("../mimic-iv-2.2/hosp/omr.csv.gz")
omr.head(10)

,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-04-27,1,Blood Pressure,110/65
1,10000032,2180-04-27,1,Weight (Lbs),94
2,10000032,2180-05-07,1,BMI (kg/m2),18.0
3,10000032,2180-05-07,1,Height (Inches),60
4,10000032,2180-05-07,1,Weight (Lbs),92.15
5,10000032,2180-05-07,2,Weight (Lbs),92.15
6,10000032,2180-05-07,3,Weight (Lbs),92.15
7,10000032,2180-05-07,4,Weight (Lbs),92.15
8,10000032,2180-05-07,5,Weight (Lbs),92.15
9,10000032,2180-05-07,6,Weight (Lbs),92.15


## `icu`

### `d_items`

In [19]:
d_items = pd.read_csv("../mimic-iv-2.2/icu/d_items.csv.gz")

In [9]:
# d_items.head()
d_items[
    d_items["label"].str.contains("Temperature", case=False, na=False)
]

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
337,223761,Temperature Fahrenheit,Temperature F,chartevents,Routine Vital Signs,°F,Numeric,NaN,NaN
338,223762,Temperature Celsius,Temperature C,chartevents,Routine Vital Signs,°C,Numeric,NaN,NaN
505,224027,Skin Temperature,Skin Temp,chartevents,Skin - Assessment,NaN,Text,NaN,NaN
767,224642,Temperature Site,Temp Site,chartevents,Routine Vital Signs,NaN,Text,NaN,NaN
790,224674,Changes in Temperature,Changes in Temperature,chartevents,Toxicology,NaN,Text,NaN,NaN
1814,226329,Blood Temperature CCO (C),Blood Temp CCO (C),chartevents,Routine Vital Signs,°C,Numeric,NaN,NaN
2097,227054,TemperatureF_ApacheIV,TemperatureF_ApacheIV,chartevents,Scores - APACHE IV (2),°F,Numeric,NaN,NaN
2776,228242,Pt. Temperature (BG) (SOFT),Pt. Temperature (BG) (SOFT),chartevents,Labs,NaN,Numeric,NaN,NaN
3466,229236,Cerebral Temperature (C),Cerebral T (C),chartevents,Hemodynamics,°C,Numeric,NaN,NaN


### `chartevents`

In [4]:
t1 = timeit(
    lambda: pd.read_csv("mimic-iv-2.2/icu/chartevents.csv.gz", nrows=100000), number = 10
)

t2 = timeit(
    lambda: pl.read_csv("mimic-iv-2.2/icu/chartevents.csv.gz", n_rows=100000), number = 10
)

print(t1,t2)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.
1.3999643280403689 1.177795070107095


In [3]:
# resave into parquet
chartevents = load_gz_csv("mimic-iv-2.2/icu/chartevents.csv.gz")
chartevents.to_parquet('mimic-iv-2.2/icu/chartevents.parquet')  

In [5]:
# eventual, much faster load:
chartevents = pd.read_parquet('../mimic-iv-2.2/icu/chartevents.parquet')

In [43]:
def find_item_label(item_id: int) -> str:
    '''
    Return the "label" string given an item_id string 
    '''
    return d_items.loc[d_items["itemid"] == item_id, "label"].values[0]

itemid_to_meas_site_name = {
    220050: "arterial",
    220179: "not specified"
}

In [44]:
find_item_label(220050)

'Arterial Blood Pressure systolic'

In [32]:
chartevents.loc[
    chartevents["itemid"] == 220050, :]

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
19182,10002013,23581541,39060235,6768.0,2160-05-18 20:00:00,2160-05-18 21:16:00,220050,116,116.0,mmHg,0.0
19327,10002013,23581541,39060235,6768.0,2160-05-18 21:00:00,2160-05-18 21:16:00,220050,97,97.0,mmHg,0.0
19353,10002013,23581541,39060235,6768.0,2160-05-18 22:00:00,2160-05-18 22:08:00,220050,123,123.0,mmHg,0.0
19376,10002013,23581541,39060235,6768.0,2160-05-18 22:10:00,2160-05-18 22:11:00,220050,94,94.0,mmHg,0.0
19385,10002013,23581541,39060235,6768.0,2160-05-18 23:00:00,2160-05-18 23:01:00,220050,116,116.0,mmHg,0.0
...,...,...,...,...,...,...,...,...,...,...,...
313630748,19999442,26785317,32336619,59028.0,2148-11-20 06:00:00,2148-11-20 06:26:00,220050,124,124.0,mmHg,0.0
313630778,19999442,26785317,32336619,59028.0,2148-11-20 20:00:00,2148-11-20 21:35:00,220050,104,104.0,mmHg,0.0
313630874,19999442,26785317,32336619,59028.0,2148-11-20 21:00:00,2148-11-20 21:35:00,220050,102,102.0,mmHg,0.0
313630901,19999442,26785317,32336619,59028.0,2148-11-20 22:00:00,2148-11-20 23:48:00,220050,105,105.0,mmHg,0.0


In [ ]:
def map_one_item(item_id):
    '''
    create the rows in a vitals table corresponding to one item from MIMIC, based on item id.
    '''


In [47]:
item_id = 220050
vital_name = find_item_label(item_id)
vital_category = "sbp"
meas_site_name = itemid_to_meas_site_name[item_id]

vitals_sbp = (
    chartevents
    .loc[
        chartevents["itemid"] == item_id,  # :
        ["hadm_id", "charttime", "itemid", "value"]
    ]
    .rename(columns = {
        "hadm_id": "encounter_id", 
        "charttime": "recorded_dttm",
        "value": "vital_value"
        })
    .assign(
        vital_name = vital_name,
        vital_category = vital_category,
        meas_site_name = meas_site_name
    )
    .reindex(
        columns = ["encounter_id", "recorded_dttm", "vital_name", "vital_category", "vital_value", "meas_site_name"]
    )
)


In [48]:
vitals_sbp

,encounter_id,recorded_dttm,vital_name,vital_category,vital_value,meas_site_name
19182,23581541,2160-05-18 20:00:00,Arterial Blood Pressure systolic,sbp,116,arterial
19327,23581541,2160-05-18 21:00:00,Arterial Blood Pressure systolic,sbp,97,arterial
19353,23581541,2160-05-18 22:00:00,Arterial Blood Pressure systolic,sbp,123,arterial
19376,23581541,2160-05-18 22:10:00,Arterial Blood Pressure systolic,sbp,94,arterial
19385,23581541,2160-05-18 23:00:00,Arterial Blood Pressure systolic,sbp,116,arterial
...,...,...,...,...,...,...
313630748,26785317,2148-11-20 06:00:00,Arterial Blood Pressure systolic,sbp,124,arterial
313630778,26785317,2148-11-20 20:00:00,Arterial Blood Pressure systolic,sbp,104,arterial
313630874,26785317,2148-11-20 21:00:00,Arterial Blood Pressure systolic,sbp,102,arterial
313630901,26785317,2148-11-20 22:00:00,Arterial Blood Pressure systolic,sbp,105,arterial
